In [ ]:
import os
import glob
from shutil import copyfile
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
%cd drive/MyDrive/The_Turing_Lens
%ls

/content/drive/MyDrive/The_Turing_Lens
'Colab Notebooks'/   data/   data_unused/   data_v1/   Milestone_0311.gdoc


In [ ]:
file_type = [".jpg", '.png', '.webp', 'jpeg']
folder_name = ['bing', 'midjourney', 'sd', 'portrait']
dict_folder_name = {'bing':'3_bing', 'midjourney':'1_midjourney', 'sd':'2_SD', 'portrait':'0_portrait'}
df = pd.DataFrame(columns=['source', 'filename', 'split'])
for name in folder_name:
    file = []
    for accept_type in file_type:
        folder_path = os.path.join('data', name, '*'+accept_type)
        file += glob.glob(os.path.join(folder_path))

    # 將資料輸入df中
    df_temp = []
    for f in file:
        parts = f.split('/')
        df_temp.append(
            dict(
                source=dict_folder_name[parts[-2]],
                filename=f
            )
        )
    df_temp = pd.DataFrame(df_temp)
    # Sample
    # 依照人工設定規則： [train:val:test = 400:40:40]/bing維持[200:20:20]
    if name == 'bing':
      n_train = 200
      n_val = 20
      n_test = 20
    elif name != 'bing':
      n_train = 400
      n_val = 40
      n_test = 40
    df_temp["split"] = 'flag'
    df_temp.loc[df_temp.query("split == 'flag'").sample(n_train, random_state=42).index, "split"] = "train"
    df_temp.loc[df_temp.query("split == 'flag'").sample(n_val, random_state=42).index, "split"] = "val"
    df_temp.loc[df_temp.query("split == 'flag'").sample(n_test, random_state=42).index, "split"] = "test"
    df_temp = df_temp[df_temp['split']!='flag']
    df = pd.concat([df, df_temp], axis=0)
df

,source,filename,split
0,3_bing,data/bing/_05f5f904-e0f1-470d-bdff-6df89c10db5...,train
1,3_bing,data/bing/_2a6a73e6-5950-4714-aaa6-01adb6764e7...,val
2,3_bing,data/bing/_e0a4492b-c156-46c6-a9af-fa52134632a...,train
3,3_bing,data/bing/OIG4.gQ9y.o5JXXrks4CJwL.jpg,train
4,3_bing,data/bing/01.jpg,train
...,...,...,...
576,0_portrait,data/portrait/unsplash_0309_221.jpg,train
578,0_portrait,data/portrait/unsplash_0309_232.jpg,train
579,0_portrait,data/portrait/unsplash_0309_84.jpg,train
581,0_portrait,data/portrait/unsplash_0309_115.jpg,train


In [ ]:
df['split'].value_counts()

train    1400
val       140
test      140
Name: split, dtype: int64

In [ ]:
df['source'].value_counts()

1_midjourney    480
2_SD            480
0_portrait      480
3_bing          240
Name: source, dtype: int64

In [ ]:
# Create a dataset
export_dst = "data_v2"
for i in range(len(df)):
    source, split, filename = df.iloc[i][["source", "split", "filename"]]
    os.makedirs(os.path.join(export_dst, split, source), exist_ok=True)
    copyfile(filename, os.path.join(export_dst, split, source, os.path.basename(filename)))